In [3]:
!pip install -q accelerate bitsandbytes transformers unsloth datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments,DataCollatorForLanguageModeling
from datasets import load_dataset
from trl import SFTTrainer
import torch
import os


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("json",data_files="exprep.jsonl",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
def format_to_chat(example):
  if example["input"].strip():
    messages = [
         {"role": "user", "content": f"{example['instruction']}\n\n{example['input']}"},
         {"role": "assistant", "content": example["output"]}
    ]
  else:
    messages = [
         {"role": "user", "content": example["instruction"]},
         {"role": "assistant", "content": example["output"]}

    ]
  return {"messages":messages}

dataset = dataset.map(format_to_chat)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
tokenizer.chat_template = (
    "{% for message in messages %}"
    "{% if message['role'] == 'user' %}"
    "{{ bos_token + 'User: ' + message['content'] + eos_token }}"
    "{% elif message['role'] == 'assistant' %}"
    "{{ bos_token + 'Assistant: ' + message['content'] + eos_token }}"
    "{% endif %}"
    "{% endfor %}"
)

In [9]:
dataset = dataset.map(lambda example: {
    "text":tokenizer.apply_chat_template(example["messages"],tokenize=False)
})

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
dataset = dataset.map(lambda x: tokenizer(x["text"]),batched=False)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [11]:
args = TrainingArguments(
    output_dir = "llama_qa_model",
    per_device_train_batch_size = 1,
    num_train_epochs = 1,
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy = "no",
    optim = "adamw_8bit",
    fp16 = True,

)

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none"
)

Unsloth 2025.7.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [14]:
os.environ["WANDB_MODE"] = "disabled"

In [15]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset = dataset,
    dataset_text_field = "input_ids",
    max_seq_length = 512,
    args = args,
    packing = False,
    data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.423000
2,4.769400
3,2.083500
4,2.461900
5,2.139800
6,1.726200
7,1.888500
8,1.701500
9,2.263900
10,1.851400


TrainOutput(global_step=10, training_loss=2.330919647216797, metrics={'train_runtime': 15.6149, 'train_samples_per_second': 0.64, 'train_steps_per_second': 0.64, 'total_flos': 27530973413376.0, 'train_loss': 2.330919647216797})

In [16]:
model.save_pretrained("llam_qa_model")
tokenizer.save_pretrained("llama_qa_model")

('llama_qa_model/tokenizer_config.json',
 'llama_qa_model/special_tokens_map.json',
 'llama_qa_model/chat_template.jinja',
 'llama_qa_model/tokenizer.json')

In [22]:
prompt = "User: Define work sampling.\nAssistant:"
inputs = tokenizer(prompt,return_tensors="pt").to("cuda")
outputs = model.generate(**inputs,max_new_tokens=100,do_sample=True,top_k=50)
print(tokenizer.decode(outputs[0],skip_special_tokens = True))

User: Define work sampling.
Assistant: Work sampling is a technique to estimate the total amount of work done by a group of workers.
